# Introduction

Now we have an idea of three important components to analyzing neuroimaging data:

1. Data manipulation
2. Cleaning and confound regression
3. Parcellation and signal extraction

In this notebook the goal is to integrate these 3 basic components and perform a full analysis of one subject using **Functional Connectivity (FC)**.

ROI-based correlational analysis forms the basis of many more sophisticated kinds of functional imaging analysis.

<div class="alert alert-block alert-success">
<b>Exercise 0 </b><p>
Before we start. What do we need to do with the fMRI data before we can do the analysis?<p>

Please explain these terms (group work)- what do they mean and why do we need to do these procedures?
-    Brain extraction
-    Motion correction
-    Susceptibility Distortion Correction
-    Coregistration
-    Slice Timing Correction
-    Normalization
-    Confound regression
-    Temporal and spatial filtering


What tools can be used to do these procedures? <p>
 -End of exercise-
    </div>

### Using Nilearn's High-level functionality to compute correlation matrices

Nilearn has a built in function for extracting timeseries from functional files and doing all the extra signal processing at the same time. Let's walk through how this is done

First we'll grab our imports as usual

In [ ]:
from nilearn import image as nimg
from nilearn import plotting as nplot
import numpy as np
import pandas as pd
from bids import BIDSLayout
from nilearn import datasets
import matplotlib.pyplot as plt
import pandas

Let's grab the data that we want to perform our connectivity analysis on using PyBIDS:


In [ ]:
dataset = datasets.fetch_atlas_harvard_oxford('cort-maxprob-thr25-2mm')
atlas_filename = dataset.maps
labels = dataset.labels

Plotting can then be done as:

In [ ]:
from nilearn import plotting
plotting.plot_roi(atlas_filename)

In [ ]:
# One subject of fmri data
data = datasets.fetch_development_fmri(n_subjects=1)
func_file = data.func[0]
confound_file = data.confounds[0]

<div class="alert alert-block alert-success">
<b>Exercise 1 </b><p>
We will now look at the confound file. What do you see here? Please explain the data structure and the column values
</div>

In [ ]:
data=pandas.read_csv(confound_file, sep='\t')
data

<div class="alert alert-block alert-success">
-End of Exercise-
            </div>

Now we'll import a package from <code>nilearn</code>, called <code>input_data</code> which allows us to pull data using the parcellation file, and at the same time applying data cleaning!

We first create an object using the parcellation file <code>Harvard-Oxford atlas</code> and our cleaning settings which are the following:

Settings to use:
- Confounds: trans_x, trans_y, trans_z, rot_x, rot_y, rot_z, white_matter, csf, global_signal
- Temporal Derivatives: Yes
- high_pass = 0.009
- low_pass = 0.08
- detrend = True
- standardize = True

<div class="alert alert-block alert-success">
<b>Exercise 2 </b><p>
What are confounds, temporal derivatives, high_pass/low_pass, detrend and standardize?<p>
    -End of Exercise-
</div>

In [ ]:
from nilearn.input_data import NiftiLabelsMasker
masker = NiftiLabelsMasker(labels_img=atlas_filename,
                                      standardize=True,
                                      memory='nilearn_cache',
                                      verbose=1,
                                      detrend=True,
                                     low_pass = 0.08,
                                     high_pass = 0.009,
                                      t_r=2
                                    )

The object <code>masker</code> is now able to be used on *any functional image of the same size*. The `input_data.NiftiLabelsMasker` object is a wrapper that applies parcellation, cleaning and averaging to an functional image. For example let's apply this to our first subject:

### Using the masker
Finally with everything set up, we can now use the masker to perform our:
1. Confounds cleaning
2. Parcellation
3. Averaging within a parcel
All in one step!


In [ ]:
#Apply cleaning, parcellation and extraction to functional data
time_series = masker.fit_transform(func_file, confounds=confound_file)

In [ ]:
plt.plot(time_series[:,1])

<div class="alert alert-block alert-success">
<b>Exercise 3 </b><p>
What happens if you do not include confounds and no filter? Try it out!
</div>

In [ ]:
# Fill in the function!
masker = NiftiLabelsMasker(labels_img=atlas_filename)
time_series_noconfounds = masker.fit_transform(...)
plt.plot(time_series_noconfounds[:,1])

<div class="alert alert-block alert-success">
-End of Exercise-
            </div>

<div class="alert alert-block alert-success">
<b>Exercise 4 </b><p>
Now let's look at the time series structure. What do these two values (168,48) mean?
</div>

In [ ]:
time_series.shape

The result of running <code>masker.fit_transform</code> is a matrix that has:
- Rows matching the XXXX (168)
- Columns matching the XXXX (48)

<div class="alert alert-block alert-success">
-End of Exercise-
            </div>


**But wait!**

We originally had **49 ROIs**, what happened to 1 of them? It turns out that <code>masker</code> drops ROIs that are empty (i.e contain no brain voxels inside of them), this means that 1 of our atlas' parcels did not correspond to any region with signal! To see which ROIs are kept after computing a parcellation you can look at the <code>labels_</code> property of <code>masker</code>:

In [ ]:
print(masker.labels_)
print("Number of labels", len(masker.labels_))

If you want to work with multiple subjects (all of whom might have different missing values), you should create an array that contains the all regions (and has zeros for the missing values). This is not part of our class today. 

### Calculating Connectivity


<div class="alert alert-block alert-success">
<b>Exercise 5 </b><p>

There are different types of connectivity that we will discuss:
    1. ROI-based connectivity
    2. Independent component analysis
Please read this paper: http://www.ajnr.org/content/early/2018/01/18/ajnr.A5527 and explain the difference between these two. 
    
    
-End of Exercise-  
</div>

In our seminar today, we will focus on the ROI-based connectivity. We will calculate a *full connectivity matrix* by computing the correlation between *all pairs of ROIs* in our parcellation scheme! 

We'll use another nilearn tool called <code>ConnectivityMeasure</code> from <code>nilearn.connectome</code>. This tool will perform the full set of pairwise correlations for us

Like the masker, we need to make an object that will calculate connectivity for us.

In [ ]:
from nilearn.connectome import ConnectivityMeasure

In [ ]:
correlation_measure = ConnectivityMeasure(kind='correlation')

<div class="alert alert-block alert-success">
<b>Exercise 6 </b><p>

Try using <code>SHIFT-TAB</code> in the code line above to see what options you can put into the <code>kind</code> argument of <code>ConnectivityMeasure</code>

-End of Exercise-  
</div>

Then we use <code>correlation_measure.fit_transform()</code> in order to calculate the full correlation matrix for our parcellated data!


<div class="alert alert-block alert-success">
<b>Exercise 7 </b><p>

Please now include the missing variable in the code below to calculate the connectivity. 

</div>

In [ ]:
full_correlation_matrix = correlation_measure.fit_transform([time_series])

<div class="alert alert-block alert-success">
-End of Exercise-  
</div>

<div class="alert alert-block alert-success">
<b>Exercise 8 </b><p>
Let's now look at the shape of the resulting connectivity matrix. What does it mean? 

In [ ]:
full_correlation_matrix.shape

<div class="alert alert-block alert-success">
-End of Exercise-  
</div>

<div class="alert alert-block alert-success">
<b>Exercise 9 </b><p>

Below you find a visualization of the connectivity matrix. What do you notice? 
</div>

In [ ]:
plt.imshow(np.squeeze(full_correlation_matrix))

<div class="alert alert-block alert-success">
-End of Exercise-  
</div>

<div class="alert alert-block alert-success">
<b>Exercise 10 </b><p>

Now it is your turn! Please create a script to calculate and visualize the connectivity of the time series without removal of the confounds. What do you notice?</div>

In [ ]:
### Your code here...

<div class="alert alert-block alert-success">
-End of Exercise-  
</div>

<div class="alert alert-block alert-success">
<b>Exercise 11 </b><p>

Now that you know functional connectivity. Can you explain dynamic functional connectivity? What are its advantages and disadvantages?

-End of Exercise-  
</div>

## Congratulations!

Hopefully now you understand that:

1. fMRI data needs to be pre-processed before analyzing
2. Manipulating images in python is easily done using `nilearn` and `nibabel`
3. You can also do post-processing like confound/nuisance regression using `nilearn`
4. Parcellating is a method of simplifying and "averaging" data. The type of parcellation reflect assumptions you make about the structure of your data
5. Functional Connectivity is really just time-series correlations between two signals!

## More information: 
https://nilearn.github.io/user_guide.html 

https://conp-pcno-training.github.io/neuroimaging-carpentry/

Master theses and student assistant jobs can be made available: research@xenia-kobeleva.com